In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.0+cu118
Torchvision version: 0.15.1+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ps4burg0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ps4burg0
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 567662b0fd33ca4b022d94d3b8de896628cd32dd
  Preparing metadata (setup.py) ... done
mkdir: cannot create directory ‘images’: File exists
--2023-04-26 09:34:25--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw

In [ ]:
!pip install monai einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import monai

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
torch.save(sam.image_encoder.state_dict(),'sam_image_encoder.pth')

In [ ]:
from segment_anything.modeling.image_encoder import ImageEncoderViT
from functools import partial
from segment_anything.utils.transforms import ResizeLongestSide

In [ ]:
prompt_embed_dim = 256
image_size = 1024
vit_patch_size = 16
encoder_depth=32
encoder_embed_dim=1280
encoder_num_heads=16
encoder_global_attn_indexes=[7, 15, 23, 31]

image_encoder=ImageEncoderViT(
            depth=encoder_depth,
            embed_dim=encoder_embed_dim,
            img_size=image_size,
            mlp_ratio=4,
            norm_layer=partial(torch.nn.LayerNorm, eps=1e-6),
            num_heads=encoder_num_heads,
            patch_size=vit_patch_size,
            qkv_bias=True,
            use_rel_pos=True,
            global_attn_indexes=encoder_global_attn_indexes,
            window_size=14,
            out_chans=prompt_embed_dim,
        )


In [ ]:
model_state=torch.load('sam_image_encoder.pth')
image_encoder.load_state_dict(state_dict=model_state)

<All keys matched successfully>

In [ ]:
model = monai.networks.nets.UNETR(
    in_channels=3,
    out_channels=2,
    img_size=1024,
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=24,
    pos_embed="conv",
    norm_name="batch",
    res_block=True,
    dropout_rate=0.15,
    spatial_dims=2,
)

In [ ]:
model.vit

ViT(
  (patch_embedding): PatchEmbeddingBlock(
    (patch_embeddings): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (dropout): Dropout(p=0.15, inplace=False)
  )
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (mlp): MLPBlock(
        (linear1): Linear(in_features=768, out_features=3072, bias=True)
        (linear2): Linear(in_features=3072, out_features=768, bias=True)
        (fn): GELU(approximate='none')
        (drop1): Dropout(p=0.15, inplace=False)
        (drop2): Dropout(p=0.15, inplace=False)
      )
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): SABlock(
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (qkv): Linear(in_features=768, out_features=2304, bias=False)
        (input_rearrange): Rearrange('b h (qkv l d) -> qkv b l h d', qkv=3, l=24)
        (out_rearrange): Rearrange('b h l d -> b l (h d)')
        (drop_output): Dropout(p=0.15, inplace=False)
        (drop_weights): 

In [ ]:
image_encoder

ImageEncoderViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
  )
  (blocks): ModuleList(
    (0-31): 32 x Block(
      (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1280, out_features=3840, bias=True)
        (proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): MLPBlock(
        (lin1): Linear(in_features=1280, out_features=5120, bias=True)
        (lin2): Linear(in_features=5120, out_features=1280, bias=True)
        (act): GELU(approximate='none')
      )
    )
  )
  (neck): Sequential(
    (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): LayerNorm2d()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): LayerNorm2d()
  )
)

In [ ]:
from torch.nn import functional as F
def preprocess(x: torch.Tensor) -> torch.Tensor:
    """Normalize pixel values and pad to a square input."""
    # Normalize colors
    #x = (x - self.pixel_mean) / self.pixel_std

    # Pad
    h, w = x.shape[-2:]
    padh = image_encoder.img_size - h
    padw = image_encoder.img_size - w
    x = F.pad(x, (0, padw, 0, padh))
    return x

In [ ]:
import torchvision.transforms as T
image = cv2.imread('images/truck.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

transform=ResizeLongestSide(image_encoder.img_size)

input_image = transform.apply_image(image)
input_image_torch = torch.as_tensor(input_image,dtype=torch.float32)
input_image_torch = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]
image_transformed=preprocess(input_image_torch)

In [ ]:
image_transformed.shape

torch.Size([1, 3, 1024, 1024])

In [ ]:
image_encoder(image_transformed)

In [ ]:
predictor.get_image_embedding().shape

torch.Size([1, 256, 64, 64])